# What's in an Avocado Toast: A Supply Chain Analysis

You're in London, making an avocado toast, a quick-to-make dish that has soared in popularity on breakfast menus since the 2010s. A simple smashed avocado toast can be made with five ingredients: one ripe avocado, half a lemon, a big pinch of salt flakes, two slices of sourdough bread and a good drizzle of extra virgin olive oil. It's no small feat that most of these ingredients are readily available in grocery stores. 

In this project, you'll conduct a supply chain analysis of three of these ingredients used in an avocado toast, utilizing the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are, fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple categories tags in the `categories_tags` column.
- There is a column in each CSV file called `origins_tags` with strings for country of origin of that item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin, and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

![](avocado_wallpaper.jpeg)

In [105]:
import pandas as pd

# read tab-limited data in csv file 
avocado = pd.read_csv('data/avocado.csv', sep='\t') 

# subset columns to keep only relevant columns 
subset_cols = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags'] 
avocado = avocado[subset_cols] 

# Gather relevant categories data for avocados 
with open('data/relevant_avocado_categories.txt', 'r') as file: 
    relevant_avocado_categories = file.read().split() 
    # relevant_avocado_categories = file.read().splitlines()
    
print('avocado relevant categories: \n', relevant_avocado_categories)

## Filter data using relevant categories 

avocado.dropna(subset=['categories_tags'], inplace=True)
# transform comma separated string values in column into a list of tag words 
avocado.categories_tags = avocado.categories_tags.str.split(',') 

# Filter rows where categories_tags have any of the tags listed in relevant categories
avocado = avocado[avocado.categories_tags.apply(lambda x: bool(set(x) & set(relevant_avocado_categories)))] 
# avocado = avocado[avocado.categories_tags.apply(lambda x: any([i for i in x if i in relevant_avocado_categories]))]



## Where do most avocados come from? 

# Filter DataFrame for UK data 
avocado_uk = avocado.query("countries == 'United Kingdom'")

# Find most common country for avocado origin &  format to strip additional string characters  
top_avocado_origin = avocado_uk.origins_tags.value_counts().index[0].lstrip('en:').replace('-', ' ') 
print('avocado top origin country:', top_avocado_origin, '\n') 


# Define a function to do the above tasks on other ingredients & return its top origin country 
def read_and_filter_data(csvfile, txtfile): 
    df = pd.read_csv('data/' + csvfile, sep = '\t') 
    subset_cols = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags'] 
    df = df[subset_cols] 
    
    with open('data/'+txtfile, mode='r') as file: 
        relevant_categories = file.read().split() 
    print(csvfile[:-4], 'relevant categories: \n', relevant_categories) 
    df = df.dropna(subset=['categories_tags']) 
    df.categories_tags = df.categories_tags.str.split(',') 
    df = df[df.categories_tags.apply(lambda x: bool(set(x) & set(relevant_categories)) ) ] 
    df_uk = df.query("countries == 'United Kingdom' ") 
    top_ingredient_origin = df_uk.origins_tags.value_counts().index[0].lstrip('en:').replace('-', ' ') 
    print(csvfile[:-4], 'top origin country:', top_ingredient_origin, '\n') 
    return top_ingredient_origin

# get top origin country for olive oil & sourdough using above function 
top_olive_oil_origin = read_and_filter_data('olive_oil.csv', 'relevant_olive_oil_categories.txt') 
top_sourdough_origin = read_and_filter_data('sourdough.csv', 'relevant_sourdough_categories.txt') 



avocado relevant categories: 
 ['en:avocadoes', 'en:avocados', 'en:fresh-foods', 'en:fresh-vegetables', 'en:fruchte', 'en:fruits', 'en:raw-green-avocados', 'en:tropical-fruits', 'en:tropische-fruchte', 'en:vegetables-based-foods', 'fr:hass-avocados']
avocado top origin country: peru 

olive_oil relevant categories: 
 ['ar:huile-d-olive', 'ar:oil', 'bg:green-olive-paste', 'de:ol', 'en:aceites-de-oliva', 'en:aceites-de-oliva-virgen', 'en:aceites-de-oliva-virgen-extra', 'en:aceites-vegetales', 'en:aceites-y-grasas', 'en:aromatised-olive-oils', 'en:blend-of-olive-oils', 'en:combined-oil', 'en:combined-oil-blended-vegetable-oils', 'en:cooking-oil', 'en:extra-virgin-olive-oils', 'en:extra-virgin-olive-oils-with-garlic', 'en:extra-virgin-olive-oils-with-truffles', 'en:falvoured-olive-oils', 'en:flavored-oils', 'en:flavoured-extra-virgin-olive-oils', 'en:flavoured-oils', 'en:flavoured-olive-oil-sprays', 'en:flavoured-olive-oils', 'en:fruit-and-fruit-seed-oils', 'en:greek-cretan-kolymbari-regio